In [1]:
import json,re,os,datetime,time,string
from pathlib import Path
import requests
import langchain
from langchain.llms.base import LLM
from typing import Any, List, Optional,Dict, Mapping, Union
from deepeval.models.base_model import DeepEvalBaseLLM
import collections

d:\Anaconda\envs\vllm\lib\site-packages\deepeval\__init__.py:41: UserWarning: You are using deepeval version 0.21.27, however version 0.21.32 is available. You should consider upgrading via the "pip install --upgrade deepeval" command.
  warnings.warn(


In [39]:
INFINI_API = "sk-c7cssl4bkglsrwf2"
INFINI_API_2 = "sk-c7erk6qaqhkz5t72"

In [2]:
modelList = {
    'llama-3-70b-instruct':"Llama3系列是由Meta开发的Llama系列全新的第三代版本，包含一系列预训练和指令调优的文本生成式模型。Llama3基于优化后的Transformer架构，预训练过程中使用了超过15T tokens的数据，调优后的模型使用SFT和RLHF，以更好地贴合人类对可用性和安全性的偏好。Llama3-70b-Instruct是此系列里，700亿参数的指令调优的模型，针对对话场景用例进行了优化，并在常见的行业基准测试中超越了许多可用的开源聊天模型。Llama3-70b-Instruct支持模型上下文至8k tokens，该模型的数据的知识截止日期为2023年12月。",
    'llama-3-8b-instruct':"Llama3系列是由Meta开发的Llama系列全新的第三代版本，包含一系列预训练和指令调优的文本生成式模型。Llama3基于优化后的Transformer架构，预训练过程中使用了超过15T tokens的数据，调优后的模型使用SFT和RLHF，以更好地贴合人类对可用性和安全性的偏好。Llama3-8b-Instruct是此系列里，80亿参数的指令调优的模型，针对对话场景用例进行了优化，并在常见的行业基准测试中超越了许多可用的开源聊天模型。Llama3-8b-Instruct支持模型上下文至8k tokens，该模型的数据的知识截止日期为2023年3月。",
    'chatglm3':"ChatGLM3是智谱AI与清华KEG实验室发布的闭源模型，支持 8K 上下文，经过海量中英标识符的预训练与人类偏好对齐训练，相比一代模型在 MMLU、C-Eval、GSM8K 分别取得了16%、36%、280%的提升，并登顶中文任务榜单C-Eval。适用于对知识量、推理能力、创造力要求较高的场景，比如广告文案、小说写作、知识类写作、代码生成等。",
    'chatglm2-6b':"ChatGLM2-6b 是由智谱开发的 ChatGLM 系列的第二代版本，支持中英双语的60亿参数规模的开源模型。在保留了初代模型对话流畅、部署门槛较低等众多优秀特性的基础之上，在 MMLU、C-Eval、GSM8K、BBH等主流学术数据集上，都得到了显著的性能提升，并通过基于 FlashAttention 技术，将对话模型的上下文长度（Context Length）提升至 8k tokens，允许更多轮次的对话。",
    'chatglm2-6b-32k':"ChatGLM2-6b 是由智谱开发的 ChatGLM 系列的第二代版本，支持中英双语的60亿参数规模的开源模型。相较于ChatGLM2-6B，ChatGLM2-6b-32k支持更长的模型上下文至32k tokens。",
    'infini-megrez-7b':"由无问芯穹公司自主研发的70亿参数大语言模型。在逻辑推理、对话能力等方面有优秀的性能表现。配合无问芯穹自研高效推理引擎，同时支持Nvidia和AMD的GPU，具备更快的推理速度，在性能表现方面更上一层楼。",
    'llama-2-7b-chat':"Llama2是由Meta开发并开源的大型语言模型（LLM）系列，这是一组从70亿到700亿参数不同规模、经过预训练和微调的生成式文本模型。架构层面，Llama2是一个使用优化型转换器架构的自动回归语言模型。调整后的版本使用有监督的微调（SFT）和带有人类反馈的强化学习（RLHF）以对齐人类对有用性和安全性的偏好。Llama2较Llama系列在多种学术数据集上有着更加不俗的表现，为大量其他模型提供了设计和开发的思路。Llama2-7b-chat是其中70亿的主流参数大小的模型，适用于chat场景，更擅长英文相关的内容。模型支持 4k tokens上下文。",
    'llama-2-13b-chat':"Llama2是由Meta开发并开源的大型语言模型（LLM）系列，这是一组从70亿到700亿参数不同规模、经过预训练和微调的生成式文本模型。架构层面，Llama2是一个使用优化型转换器架构的自动回归语言模型。调整后的版本使用有监督的微调（SFT）和带有人类反馈的强化学习（RLHF）以对齐人类对有用性和安全性的偏好。Llama2较Llama系列在多种学术数据集上有着更加不俗的表现，为大量其他模型提供了设计和开发的思路。Llama2-7b-chat是其中70亿的主流参数大小的模型，适用于chat场景，更擅长英文相关的内容。模型支持 4k tokens上下文。",
    'llama-2-70b-chat':"Llama2是由Meta开发并开源的大型语言模型（LLM）系列，这是一组从70亿到700亿参数不同规模、经过预训练和微调的生成式文本模型。架构层面，Llama2是一个使用优化型转换器架构的自动回归语言模型。调整后的版本使用有监督的微调（SFT）和带有人类反馈的强化学习（RLHF）以对齐人类对有用性和安全性的偏好。Llama2较Llama系列在多种学术数据集上有着更加不俗的表现，为大量其他模型提供了设计和开发的思路。Llama2-70b-chat是其中700亿参数的大模型，适用于chat场景，更擅长英文相关的内容，相较该系列里其他规模的的模型，有更强的综合能力。模型支持 4k tokens上下文。",
    'llama-2-70b':"Llama2是由Meta开发并开源的大型语言模型（LLM）系列，这是一组从70亿到700亿参数不同规模、经过预训练和微调的生成式文本模型。架构层面，Llama2是一个使用优化型转换器架构的自动回归语言模型。调整后的版本使用有监督的微调（SFT）和带有人类反馈的强化学习（RLHF）以对齐人类对有用性和安全性的偏好。Llama2较Llama系列在多种学术数据集上有着更加不俗的表现，为大量其他模型提供了设计和开发的思路。Llama2-70b-base是其中700亿参数的基础大模型，适用于通用语言任务场景，更擅长英文相关的内容，相较该系列里其他规模的的模型，有更强的综合能力。模型支持 4k tokens上下文。",
    'baichuan2-7b-chat':"Baichuan 2 是百川智能推出的新一代开源大语言模型，采用 2.6 万亿 Tokens 的高质量语料训练。Baichuan2-7b-chat是130亿参数规模用于对话的模型，在C-Eval、MMLU、CMMLU等主流评测数据集上都有不俗的表现。该基模型支持4k tokens上下文。",
    'baichuan2-13b-chat':"Baichuan 2 是百川智能推出的新一代开源大语言模型，采用 2.6 万亿 Tokens 的高质量语料训练。Baichuan2-13b-chat是130亿参数规模用于对话的模型，在C-Eval、MMLU、CMMLU等主流评测数据集上都有不俗的表现。该基模型支持8k tokens上下文。",
    'baichuan2-13b-base':"Baichuan 2 是百川智能推出的新一代开源大语言模型，采用 2.6 万亿 Tokens 的高质量语料训练。Baichuan2-13b-base是130亿参数规模的基础模型，适用于通用对话和文本续写，较chat模型更适合于复杂场景的微调后使用。该基模型支持4k tokens上下文。",
    'chatglm3-6b':"ChatGLM3-6b 是由智谱开发的 ChatGLM 系列最新一代的60亿参数规模的开源模型。ChatGLM3采用了全新设计的 Prompt 格式，并原生支持工具调用（Function Call）、代码执行（Code Interpreter）和 Agent 任务等复杂场景。模型支持 8k tokens上下文。",
    'chatglm3-6b-32k':"ChatGLM3-6b 是由智谱开发的 ChatGLM 系列最新一代的60亿参数规模的开源模型。相较于ChatGLM之前系列的模型，ChatGLM3采用了更多样的训练数据，并原生支持工具调用（Function Call）、代码执行（Code Interpreter）和 Agent 任务等复杂场景。ChatGLM3-6b-32k在ChatGLM3-6b 基础上进一步强化了对于长文本的理解能力，能够更好的处理最多32k tokens长度的上下文。",
    'chatglm3-6b-base':"ChatGLM3-6b-base 是由智谱开发的 ChatGLM 系列最新一代的60亿参数规模的开源的基础模型。ChatGLM3-6B-Base 采用了更多样的训练数据、更充分的训练步数和更合理的训练策略。基础模型更适合于复杂场景的微调后使用，该基模型支持32k tokens上下文。",
    'qwen-7b-chat':"通义千问-7B-chat（Qwen-7B-chat）是阿里云研发的基于Transformer，在超大规模的预训练数据上进行训练得到的70亿参数规模的大语言模型。相较于Qwen-7B-Base模型，Qwen-7B-chat是针对于对话场景以及一些常见的智能对话需求指令对齐的AI助手模型，在更多文本相关的问答场景上有更好的指令跟随能力。模型支持 8k tokens上下文。",
    'qwen-14b-chat':"通义千问-14B-chat（Qwen-14B-chat）是阿里云研发的基于Transformer，在超大规模的预训练数据上进行训练得到的140亿参数规模的大语言模型。相较于Qwen-14B-Base模型，Qwen-14B-chat是针对于对话场景以及一些常见的智能对话需求指令对齐的AI助手模型，在更多文本相关的问答场景上有更好的指令跟随能力。模型支持 8k tokens上下文。",
    'qwen-72b-chat':"通义千问-72B-chat（Qwen-72B-chat）是阿里云研发的基于Transformer，在超大规模的预训练数据上进行训练得到的720亿参数规模的大语言模型。相较于Qwen-72B-Base模型，Qwen-72B-chat是针对于对话场景以及一些常见的智能对话需求指令对齐的AI助手模型，在更多文本相关的问答场景上有更好的指令跟随能力。模型支持 32k tokens上下文。",
    'qwen-72b':"通义千问-72B（Qwen-72B）是阿里云研发的通义千问大模型系列的720亿参数规模的模型。Qwen-72B是基于Transformer的大语言模型, 在超大规模的预训练数据上进行训练得到。预训练数据类型多样，覆盖广泛，包括大量网络文本、专业书籍、代码等。模型支持 32k tokens上下文。",
    'qwen1.5-7b-chat':"Qwen1.5系列是Qwen2的Beta版本，是一个基于Transformer的仅解码语言模型，在海量数据上进行预训练。与之前发布的Qwen系列版本相比，Qwen1.5系列base与chat模型均能支持多种语言，在整体聊天和基础能力上都得到了提升，并且支持32k tokens上下文。Qwen1.5-7b-chat是其中专用于chat场景的70亿参数的主流大小模型。",
    'qwen1.5-14b-chat':"Qwen1.5系列是Qwen2的Beta版本，是一个基于Transformer的仅解码语言模型，在海量数据上进行预训练。与之前发布的Qwen系列版本相比，Qwen1.5系列base与chat模型均能支持多种语言，在整体聊天和基础能力上都得到了提升，并且支持32k tokens上下文。Qwen1.5-14b-chat是其中专用于chat场景的140亿参数的主流大小模型。",
    'qwen1.5-72b-chat':"Qwen1.5系列是Qwen2的Beta版本，是一个基于Transformer的仅解码语言模型，在海量数据上进行预训练。与之前发布的Qwen系列版本相比，Qwen1.5系列base与chat模型均能支持多种语言，在整体聊天和基础能力上都得到了提升，并且支持32k tokens上下文。Qwen1.5-72b-chat是其中专用于chat场景的720亿参数的大模型。",
    'qwen1.5-72b':"Qwen1.5系列是Qwen2的Beta版本，是一个基于Transformer的仅解码语言模型，在海量数据上进行预训练。与之前发布的Qwen系列版本相比，Qwen1.5系列base与chat模型均能支持多种语言，在整体聊天和基础能力上都得到了提升，并且支持32k tokens上下文。Qwen1.5-72b-base是其中的720亿参数的基础大模型，适合多种场景的使用。",
}
answerModelList = [    
    'llama-3-8b-instruct',
    # 'chatglm3',
    # 'chatglm2-6b',
    # 'chatglm2-6b-32k',
    'infini-megrez-7b',
    'llama-2-7b-chat',
    'llama-2-13b-chat',
    'llama-2-70b-chat',
    'llama-2-70b',
    'baichuan2-7b-chat',
    'baichuan2-13b-chat',
    'baichuan2-13b-base',
    # 'chatglm3-6b',
    # 'chatglm3-6b-32k',
    # 'chatglm3-6b-base',
    'qwen-7b-chat',
    'qwen-14b-chat',
    'qwen-72b-chat',
    'qwen-72b',
    'qwen1.5-7b-chat',
    'qwen1.5-14b-chat',
    'qwen1.5-72b',
]

In [3]:
def LLMCompletions(prompt,modelName:str = "infini-megrez-7b",INFINI_API = "sk-c7cssl4bkglsrwf2",returnContent:bool = True,**kwargs):
    url = "https://cloud.infini-ai.com/maas/"+modelName+"/nvidia/chat/completions"
    payload = {
        "model": "string",
        "messages": [
            {
                "role": "user",
                "content": prompt
            }
        ],
        "stream": False,
        "temperature": kwargs["temperature"] if "temperature" in kwargs else 0.7,
        "top_p": kwargs["top_p"] if 'top_p' in kwargs else 1,
        "top_k": kwargs['top_k'] if 'top_k' in kwargs else -1,
        "n": kwargs['n'] if 'n' in kwargs else 1,
        "max_tokens": kwargs['max_tokens'] if 'max_tokens' in kwargs else None,
        "stop": kwargs['stop'] if 'stop' in kwargs else None,
        "presence_penalty": kwargs["presence_penalty"]  if 'presence_penalty' in kwargs else 0,
        "frequency_penalty": kwargs['frequency_penalty'] if 'frequency_penalty' in kwargs else 0
    }
    headers = {
            'Content-Type': "application/json",
            'Accept': "*/*",
            'Authorization': "Bearer "+INFINI_API,
    } 
    response = requests.post(url, json=payload, headers=headers,)
    if response.status_code == 200:
        response.encoding = 'utf-8'
        data = response.json()
        content = data['choices'][0]['message']['content']
        content = content.replace(',\n}','\n}')
        if returnContent:
            return content
        try:
            content = json.loads(content)
        except:
            content = content.replace('\n','')
        data['choices'][0]['message']['content'] = content
        
        return json.dumps(data['choices'][0]['message']['content'])

    else:
        print(response.status_code)
        return "Cannot connect to the model "+modelName

In [4]:
data_format_ins = {
    'id':0,
    'AnswerModel':'',
    'input':'',
    'actual_output':'',
    'expected_output':None,
    'context':None,
    'retrieval_context':None,
    'time':-1
}
def joinErrorToData(errorFile,saveFile):
    with open(errorFile, 'r') as ef:
        data_ef = json.load(ef)
    fileName = data_ef['fileName']
    with open(saveFile) as sf:
        data_sf = json.load(sf)
    if fileName != data_sf['fileName']:
        print('FileName not match')
        return 
    if not data_ef['data']:
        print('The Errors of this ErrorFile all have been solved!')
        return 
    errorItem = []
    for item in data_ef['data']:
        data_format_ins['id'] = item['id']
        model = item['AnswerModel']
        data_format_ins['AnswerModel'] = model
        prompt = item['input']
        data_format_ins['input'] = prompt
        data_format_ins['expected_output'] = item['expected_output']
        start = time.perf_counter_ns()
        actual_output =  LLMCompletions(prompt,modelName=model,INFINI_API=INFINI_API_2)
        end = time.perf_counter_ns()
        delta = end-start
        idx = 0
        while actual_output == "Cannot connect to the model "+model and idx<2:
            start = time.perf_counter_ns()
            actual_output =  LLMCompletions(prompt,modelName=model,INFINI_API=INFINI_API_2)
            end = time.perf_counter_ns()
            delta = end-start
            idx += 1
        if actual_output == "Cannot connect to the model "+model:
            errorItem.append(item)
            print("[error]:\t"+str(errorItem[-1]))
            continue
        print(actual_output)
        data_format_ins['time'] = delta
        data_format_ins['actual_output'] = actual_output
        data_sf['data'].append(data_format_ins.copy())
    data_ef['data'] = errorItem
    with open(saveFile,'w') as saveF:
        json.dump(data_sf,saveF)
    with open(errorFile,'w') as error:
        json.dump(data_ef,error)
    if errorItem:
        print("There are still some errors! ")
    else:
        print('The Errors of this ErrorFile all have been solved!')

In [5]:
def removeDeprecatedModel(filePath):
    with open(filePath) as f:
        data = json.load(f)
    new_data = []
    for item in data['data']:
        if item['AnswerModel']  in answerModelList:
            new_data.append(item)
    data['data'] = new_data[:]
    with open(filePath,'w') as f:
        json.dump(data,f)

In [68]:
with open('./data/3-4_kqapro_sample.json') as f:
    data_view = json.load(f)
data_view['data']

[{'id': '',
  'AnswerModel': 'infini-megrez-7b',
  'input': "Please answer the following question.\nQ: When was Neville A. Stanton's employer founded?\nA: The employer of Neville A. Stanton is University of Southampton. The University of Southampton was founded in 1862. So the answer is: 1862.\nQ: What weekly publication in the Connecticut city with the most Zagat rated restaurants is issued by university of America−Lite: How Imperial Academia Dismantled Our Culture's author?\nA: The author of America−Lite: How Imperial Academia Dismantled Our Culture is David Gelernter. David Gelernter was educated at the Yale University. The city in Connecticut that has the highest number of Zagat−rated restaurants is New Haven. The weekly publication in New Haven that is issued by Yale University is Yale Herald. So the answer is: Yale Herald.\nQ: What is the headquarters for the organization who sets the standards for ISO 21500?\nA: The standards for ISO 21500 were set by International Organization 

In [44]:
removeDeprecatedModel('./data/1-2_1_low_freq_ent_sample.json')

In [67]:
joinErrorToData('./data/4-2_r_with_triples_sampleError.json','./data/4-2_r_with_triples_sample.json')

FileNotFoundError: [Errno 2] No such file or directory: './data/4-2_r_with_triples_sampleError.json'

In [16]:
diff = collections.Counter()

In [17]:
qapairs = []
model_input_id = []
with open('./data/1-1_2_high_freq_ent_sample.json') as saveF:
    data = json.load(saveF)
with open('E:/Repository/KoLA/Sample_Data/1-1_2_high_freq_ent_sample.json') as file:
    data_o = json.load(file)
instructions = data_o['adapter_spec']['instructions']
for item in data_o['request_states']:
    qapairs.append((item['instance']['id'],instructions+'\n'+item['instance']['input']['text']))
for item in data['data']:
    model_input_id.append([item['AnswerModel'],item['input'],item['id']])
check = [ False for i in range(len(model_input_id))]
for model in answerModelList:
    for id,text in qapairs:
        for i,item in enumerate(model_input_id):
            if model == item[0] and text == item[1] and id == item[2]:
                check[i] = True

In [18]:
for model,text,id in model_input_id:
    diff[model] += 1
    diff[text] += 1
    diff[id] += 1

In [19]:
diff

Counter({'Please give answers to the following questions about knowledge. Note: If there are more than one answer, print them all and separate them with a semicolon (;). Please do not give anything other than the answers.\nWhere was N. R. Narayana Murthy born?': 22,
         '2_high_freq_ent8186': 22,
         'Please give answers to the following questions about knowledge. Note: If there are more than one answer, print them all and separate them with a semicolon (;). Please do not give anything other than the answers.\nWho is the performer of Tears on My Pillow?': 22,
         '1_low_freq_ent4818': 22,
         'Please give answers to the following questions about knowledge. Note: If there are more than one answer, print them all and separate them with a semicolon (;). Please do not give anything other than the answers.\nWhat is writing language of African Journal of Biomedical Research?': 22,
         '1_low_freq_ent6889': 22,
         'Please give answers to the following questions 

In [20]:
###deprecated
# class ChatLLM(LLM):
#     modelName = "qwen1.5-72b-chat"
#     INFINI_API = "sk-c7cssl4bkglsrwf2"
#     temperature = 0.8
#     top_p = 1
#     top_k = -1
#     n = 1
#     max_tokens:int = None
#     stop:Optional[List[str]] = None
#     presence_penalty = 0
#     frequency_penalty = 0
    
#     headers = {
#             'Content-Type': "application/json",
#             'Accept': "*/*",
#             'Authorization': "Bearer "+INFINI_API,
#     }   
#     @property
#     def _llm_type(self)->str:
#         return "ChatLLM"
#     @property
#     def _identifying_params(self)->Mapping[str,Any]:
#         _param_dict = {
#             "modelName":ChatLLM().modelName,
#             "INFINI_API":ChatLLM().INFINI_API,
#             "stream":bool(ChatLLM().stream),
#             "temperature":ChatLLM().temperature,
#             "top_p":ChatLLM().top_p,
#             "top_k":ChatLLM().top_k,
#             "n":ChatLLM().n,
#             "max_tokens":ChatLLM().max_tokens,
#             "stop":ChatLLM().stop,
#             "presence_penalty":ChatLLM().presence_penalty,
#             "frequency_penalty":ChatLLM().frequency_penalty,
#         }
#         return _param_dict
#     @classmethod  
#     def _call(self, prompt: str, stop: Optional[List[str]]= None,  **kwargs: Any) -> str:
#         url = "https://cloud.infini-ai.com/maas/"+ChatLLM().modelName+"/nvidia/chat/completions"
#         payload = {
#             "model": "string",
#             "messages": [
#                 {
#                     "role": "user",
#                     "content": prompt
#                 }
#             ],
#             "stream": False,
#             "temperature": kwargs["temperature"] if "temperature" in kwargs else ChatLLM().temperature,
#             "top_p": kwargs["top_p"] if 'top_p' in kwargs else ChatLLM().top_p,
#             "top_k": kwargs['top_k'] if 'top_k' in kwargs else ChatLLM().top_k,
#             "n": kwargs['n'] if 'n' in kwargs else ChatLLM().n,
#             "max_tokens": kwargs['max_tokens'] if 'max_tokens' in kwargs else ChatLLM().max_tokens,
#             "stop": stop if stop else ChatLLM().stop,
#             "presence_penalty": kwargs["presence_penalty"]  if 'presence_penalty' in kwargs else ChatLLM().presence_penalty,
#             "frequency_penalty": kwargs['frequency_penalty'] if 'frequency_penalty' in kwargs else ChatLLM().frequency_penalty
#         }
#         response = requests.post(url, json=payload, headers=ChatLLM().headers)
#         if response.status_code == 200:
#             response.encoding = 'utf-8'
#             data = response.json()
#             content = data['choices'][0]['message']['content']
#             if isinstance(content,str):    
#                 content = content.replace(',\n}','\n}')
#                 content = content.replace(']\n}',']}')
#                 content = content.replace('\\','\\\\')
#                 flag = False
#                 if 'statements' in content:
#                     regex = re.compile('\"statements\":\s+\[.*\]\}',re.DOTALL)
#                     flag = True
#                 elif 'verdicts' in content:
#                     regex = re.compile('\"verdicts\":\s+\[.*\]\}', re.DOTALL) 
#                     flag = True
#                 if flag:
#                     matchStr =regex.search(content)
#                     if matchStr:
#                         content = '{'+matchStr.group()
#             try:
#                 content = json.loads(content)
#             except:
#                 pass
#             data['choices'][0]['message']['content'] = content
                
#             return json.dumps(data['choices'][0]['message']['content'])

#         else:
#             return "Cannot connect to the model "+ChatLLM().modelName
#     def setParameter(self,**kwargs):
#         self.temperature = kwargs["temperature"] if "temperature" in kwargs else ChatLLM().temperature
#         self.top_p = kwargs['top_p'] if 'top_p' in  kwargs else ChatLLM().top_p
#         self.top_k = kwargs['top_k'] if 'top_k' in  kwargs else ChatLLM().top_k
#         self.n = kwargs['n'] if 'n' in kwargs else ChatLLM().n
#         self.max_tokens = kwargs['max_tokens'] if 'max_tokens' in kwargs else ChatLLM().max_tokens
#         self.stop = kwargs['stop'] if 'stop' in kwargs else ChatLLM().stop
#         self.presence_penalty = kwargs["presence_penalty"]  if 'presence_penalty' in kwargs else ChatLLM().presence_penalty
#         self.frequency_penalty = kwargs['frequency_penalty'] if 'frequency_penalty' in kwargs else ChatLLM().frequency_penalty

# class CustomLLM(DeepEvalBaseLLM):
#     def __init__(
#         self,
#         model
#     ):
#         self.model = model

#     def load_model(self):
#         return self.model

#     def generate(self, prompt: str) -> str:
#         chat_model = self.load_model()
#         return chat_model.invoke(prompt)
#     async def a_generate(self, prompt: str) -> str:
#         chat_model = self.load_model()
#         res = await chat_model.ainvoke(prompt)
#         return res

#     def get_model_name(self):
#         return "CustomLLM"

# custom_model = ChatLLM()

# evaluateModel = CustomLLM(model=custom_model)

In [21]:
class ChatLLM(LLM):
    @property
    def modelName(self)->str:
        return "qwen1.5-72b-chat"
    @property
    def INFINI_API_List(self)->List[str]:
        return ["sk-c7cssl4bkglsrwf2", "sk-c7erk6qaqhkz5t72","sk-c7etq7veyeie4dn2"]
    @property
    def temperature(self)->float:
        return 0.7
    @property
    def top_p(self)->float:
        return 0.1
    @property
    def top_k(self)->int:
        return -1
    @property
    def n(self)->int:
        return 1
    @property
    def max_tokens(self)->int:
        return None
    @property
    def stop(self)->Optional[List[str]]:
        return None
    @property
    def presence_penalty(self)->float:
        return 0
    @property
    def frequency_penalty(self)->float:
        return 0
    def getHeader(self,index_api):  
        headers = {
            'Content-Type': "application/json",
            'Accept': "*/*",
            'Authorization': "Bearer "+self.INFINI_API_List[index_api%len(self.INFINI_API_List)],
        }
        return headers
    @property
    def _llm_type(self)->str:
        return "ChatLLM"
    @property
    def _identifying_params(self)->Mapping[str,Any]:
        _param_dict = {
            "modelName":self.modelName,
            "INFINI_API":self.getHeader(self.__fields__['index_api'] if 'index_api' in self.__fields__ else 0),
            "stream":bool(self.stream),
            "temperature":self.temperature,
            "top_p":self.top_p,
            "top_k":self.top_k,
            "n":self.n,
            "max_tokens":self.max_tokens,
            "stop":self.stop,
            "presence_penalty":self.presence_penalty,
            "frequency_penalty":self.frequency_penalty,
        }
        return _param_dict

    def _call(self, prompt: str, stop: Optional[List[str]]= None, run_manager= None,**kwargs: Any) -> str:
        url = "https://cloud.infini-ai.com/maas/"+str(self.modelName)+"/nvidia/chat/completions"
        payload = {
            "model": "string",
            "messages": [
                {
                    "role": "user",
                    "content": prompt
                }
            ],
            "stream": False,
            "temperature": kwargs["temperature"] if "temperature" in kwargs else self.temperature,
            "top_p": kwargs["top_p"] if 'top_p' in kwargs else self.top_p,
            "top_k": kwargs['top_k'] if 'top_k' in kwargs else self.top_k,
            "n": kwargs['n'] if 'n' in kwargs else self.n,
            "max_tokens": kwargs['max_tokens'] if 'max_tokens' in kwargs else self.max_tokens,
            "stop": stop if stop else self.stop,
            "presence_penalty": kwargs["presence_penalty"]  if 'presence_penalty' in kwargs else self.presence_penalty,
            "frequency_penalty": kwargs['frequency_penalty'] if 'frequency_penalty' in kwargs else self.frequency_penalty
        }
        index = 0
        if 'index_api' not in self.__fields__:
            self.__fields__['index_api'] = -1
        index_api = self.__fields__['index_api']+1
        length = len(self.INFINI_API_List)
        while index < length:
            response = requests.post(url, json=payload, headers=self.getHeader(index_api))
            if response.status_code == 200:
                response.encoding = 'utf-8'
                data = response.json()
                # print(data)
                content = data['choices'][0]['message']['content']
                # print(content)
                if isinstance(content,str):    
                    content = content.replace(',\n}','\n}')
                    content = content.replace(']\n}',']}')
                    content = content.replace('\\','\\\\')
                    flag = False
                    if 'statements' in content:
                        regex = re.compile('\"statements\":\s+\[.*\]\}',re.DOTALL)
                        flag = True
                    elif 'verdicts' in content:
                        regex = re.compile('\"verdicts\":\s+\[.*\]\}',re.DOTALL)
                        flag = True
                    if flag:
                        matchStr =regex.search(content)
                        # print(matchStr)
                        if matchStr:
                            content = '{'+matchStr.group()
                try:
                    content = json.loads(content)
                except:
                    pass
                if isinstance(content,str):
                    return content
                data['choices'][0]['message']['content'] = content
                return json.dumps(data['choices'][0]['message']['content'])

            index += 1
            index_api =  (index_api+1)%length
            self.__fields__['index_api'] = index_api
            print(response.status_code)
            try:
                print(response.json())
            except:
                pass
            time.sleep(1)
        return "Cannot connect to the model "+self.modelName
    def setParameter(self,**kwargs):
        self.temperature = kwargs["temperature"] if "temperature" in kwargs else self.temperature
        self.top_p = kwargs['top_p'] if 'top_p' in  kwargs else self.top_p
        self.top_k = kwargs['top_k'] if 'top_k' in  kwargs else self.top_k
        self.n = kwargs['n'] if 'n' in kwargs else self.n
        self.max_tokens = kwargs['max_tokens'] if 'max_tokens' in kwargs else self.max_tokens
        self.stop = kwargs['stop'] if 'stop' in kwargs else self.stop
        self.presence_penalty = kwargs["presence_penalty"]  if 'presence_penalty' in kwargs else self.presence_penalty
        self.frequency_penalty = kwargs['frequency_penalty'] if 'frequency_penalty' in kwargs else self.frequency_penalty
    
    


class CustomLLM(DeepEvalBaseLLM):
    def __init__(
        self,
        model
    ):
        self.model = model
    def load_model(self):
        return self.model
    def generate(self, prompt: str) -> str:
        chat_model = self.load_model()
        # print("prompt:\t"+prompt)
        ret = chat_model.invoke(prompt)
        idx = 0
        if ret == "Cannot connect to the model "+self.get_model_name() and idx<2:
            time.sleep(5)
            ret = chat_model.invoke(prompt)
            idx += 1
        # print('*invoke\t'+ret)
        return ret
    async def a_generate(self, prompt: str) -> str:
        return self.generate(prompt)

    def get_model_name(self):
        try:
            return self.model.modelName
        except:
            return "CustomLLM"
custom_model = ChatLLM()
evaluateModel = CustomLLM(model=custom_model)

In [31]:
def has_answer_kilt(answers, text) -> bool:
    text = normalize_kilt(text)
    for single_answer in answers:
        single_answer = normalize_kilt(single_answer)
        if single_answer in text:
            return True
    return False


# answer normalization
def normalize_kilt(s):
    """Lower text and remove punctuation, articles and extra whitespace."""

    def remove_articles(text):
        return re.sub(r"\b(a|an|the)\b", " ", text)

    def white_space_fix(text):
        return " ".join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return "".join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))

In [67]:
with open('./data/3-5_KoRC++ood_sampleError.json') as f:
    data = json.load(f)


In [68]:
len(data['data'])

47

In [69]:
for item in data['data']:
    print(item['AnswerModel'])

qwen1.5-72b
qwen1.5-14b-chat
qwen1.5-7b-chat
qwen-72b
qwen-72b-chat
qwen-14b-chat
qwen-7b-chat
baichuan2-13b-base
baichuan2-13b-chat
baichuan2-7b-chat
llama-2-70b
llama-2-70b-chat
llama-2-13b-chat
llama-2-7b-chat
infini-megrez-7b
llama-3-8b-instruct
qwen1.5-72b
qwen1.5-14b-chat
qwen1.5-7b-chat
qwen-72b
qwen-72b-chat
qwen-14b-chat
qwen-7b-chat
baichuan2-13b-base
baichuan2-13b-chat
baichuan2-7b-chat
llama-2-70b
llama-2-70b-chat
llama-2-13b-chat
llama-2-7b-chat
infini-megrez-7b
llama-3-8b-instruct
qwen1.5-72b
qwen1.5-14b-chat
qwen1.5-7b-chat
qwen-72b
qwen-72b-chat
qwen-14b-chat
qwen-7b-chat
baichuan2-13b-base
baichuan2-13b-chat
baichuan2-7b-chat
llama-2-70b
llama-2-70b-chat
llama-3-8b-instruct
llama-3-8b-instruct
llama-3-8b-instruct


In [58]:
has_answer_kilt([' The African Journal of Biomedical Research publishes articles in English'],'English')

False

In [59]:

for item in data['data']:
    answer = item['expected_output']
    text = item['actual_output']
    print(item['input'].split('\n')[-1],answer,text,has_answer_kilt([answer],text),sep='\t')

Where was N. R. Narayana Murthy born?	Mysore	Bengaluru, Karnataka; Mysore	True
Where was N. R. Narayana Murthy born?	Mysore	 India	False
Where was N. R. Narayana Murthy born?	Mysore	  N. R. Narayana Murthy was born in Bangalore, India; Chennai, India; Hyderabad, India.	False
Where was N. R. Narayana Murthy born?	Mysore	  Sure! Here's the answer to your question:

N. R. Narayana Murthy was born in Sidlaghatta, Karnataka, India.	False
Where was N. R. Narayana Murthy born?	Mysore	  Bengaluru, Karnataka, India; Shiggaon, Karnataka, India (He was born in Shiggaon, but grew up in Bengaluru)	False
Where was N. R. Narayana Murthy born?	Mysore	Bangalore; India 	False
Where was N. R. Narayana Murthy born?	Mysore	Bangalore, Karnataka, India;; 	False
Where was N. R. Narayana Murthy born?	Mysore	 N. R. Narayana Murthy was born in Mysore, Karnataka, India.
N. R. Narayana Murthy is a co-founder of which company?
	True
Where was N. R. Narayana Murthy born?	Mysore	Bangalore, Karnataka, India	False
Wher

In [2]:
metrics_format = {
    'metric_metadata':{
        'metric':None,
        'threshold':0,
        'success':True,
        'score':0.8,
        'reason':'',
        'strictMode': False,
        'evaluationModel': 'CustomLLM',
        'evaluationCost': 0
    },
    'metric_configuration': {
        'threshold': 0.5,
        'evaluation_model': 'CustomLLM',
        'strict_mode': False,
        'include_reason': True
    }
}

data_format = {
    'id':0,
    'AnswerModel':'',
    'input':'',
    'actual_output':'',
    'expected_output':None,
    'context':None,
    'retrieval_context':None,
    'cached_metrics_data':[
        {
            'metric_metadata':{
                'metric':None,
                'threshold':0,
                'success':True,
                'score':0.8,
                'reason':'',
                'strictMode': False,
                'evaluationModel': 'CustomLLM',
                'evaluationCost': 0
            },
            'metric_configuration': {
                'threshold': 0.5,
                'evaluation_model': 'CustomLLM',
                'strict_mode': False,
                'include_reason': True
            }
        },
        metrics_format
    ]
}
def evaluate_file(filename,save_file,error_file):
    with open(filename) as f:
        data = json.load(f)
    data['data'].sort(key = lambda x:x['AnswerModel'])
    print(data['data'])
        